# 📖 1. Импорт библиотек


In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
# импортируем библиотеки

In [2]:
url = 'https://www.kinonews.ru/top250imdb/?ysclid=m7mue6xnr1673496081' # переменная, содержащая в себе ссылку на сайт для парсинга

In [3]:
pages = requests.get(url) # подключаемся
soup = bs(pages.text, 'html.parser') # определеяем
print(pages.status_code) # вывод статуса состояния

200


In [4]:
result_list = {'idFilm':[], 'titleRU':[], 'titleEN':[], 'yearPub':[], 'description':[], 'chrono':[], 'country':[], 'genre':[], 'producer':[], 'starActor1':[], 'starActor2':[], 'rating':[]} # словарь для будущего датафрейма

# 2. Парсинг 📤


In [5]:
pagenum = 1 # номер страницы(изначально 1)
%time
# этот цикл для того, чтобы перемещаться по страницапм
for i in range(5):
    # Пустые переменные для записи туда значений
    idFilm = 0
    titleFilmRU = ''
    titleFilmEN = ''
    yearFilm = 0
    rating = 0
    country = ''
    genre = ''
    chrono = ''
    producer = ''
    starActor1 = ''
    starActor2 = ''
    description = ''
    # юрлка
    url = 'https://www.kinonews.ru/top250imdb_p'+ str(pagenum) + '/' # тут видно что переходим на страницу
    print(url) # вывожу чтобы видеть процесс парсинга
    kino = requests.get(url) # также подключаемся
    soup = bs(kino.text, 'html.parser') #также определяем
    
    data = soup.find('div', class_='block-page-new') # Это нахожу контейнер/карточку фильма
    titletext = data.find_all('div', class_='bigtext') # В нем нахожу главный текст
    othertext = data.find_all('div', class_='relative') # И другой текст с информацией
    
    for i in titletext: # сейчас будем заходить на все карточки фильмов по очереди
        # Для начала соберу описание, перейдя прям на страничку фильам, тк описание содержится там
        try:
            urldesc = 'https://www.kinonews.ru' + str(i.a.get('href')) # юрлка
            desc = requests.get(urldesc) # Подключаюсь
            soup2 = bs(desc.text, 'html.parser') # собсна текст сайта
            # тк в некоторых фильмах разработчик сайта решил добавлять описание 
            # то просто в текст div, то в тег p, обрабатываю этот момент
            try:
                description = soup2.find('div', itemprop="description").p.text 
            except:
                description = soup2.find('div', itemprop="description").text
        except:
            description = '-'
        
        # Нахожу id фильма в заглавном тексте
        idFilm =  int(i.find('b').text.replace('.', ''))# тут нахожу id фильма
        try:
            titleFilmRU = i.find('a').text.strip()# тут нахожу название фильма на русском
        except AttributeError:
            titleFilmRU = '-'
            
        try:
            titleFilmEN = i.find('span').text.replace('(', '').replace(')', '').strip() # на англ
            
        except AttributeError:
            titleFilmEN = '-'
        
        try:
            yearFilm = int(i.text.split(',')[-1].strip())# год выпуска фильма
        except ValueError:
            yearFilm = 1970
            
        # Далее после того как нашли данные, нужно их записать в окончательный словарь, который позже преобразуется в датафрейм
        result_list['idFilm'].append(idFilm)
        result_list['titleRU'].append(titleFilmRU)
        result_list['titleEN'].append(titleFilmEN)
        result_list['yearPub'].append(yearFilm)
        result_list['description'].append(description)
        
        
    # Теперь по доп.тексту    
    for i in othertext:
        allcountry = i.find_all('div', class_='textgray')
        # Переменная полной инфы, тк разработчик сайта сделал корявую структуру,
        # просто буду записывать всю нужную инфу в эту переменную
        fullinf = '' 
        for j in allcountry:
            z = j.text.split(':')[1].strip()
            fullinf += z + ';' #Разделяю ; для удобства
        try:
            fullinf += i.find('span', class_='rating-big').text + ';'
        except AttributeError:
            print('Error')
        try:
            # Ну и просто теперь вытаскиваю из той переменной, разделяя по ;
            country = fullinf.split(';')[0]  
            genre = fullinf.split(';')[1]
            producer = fullinf.split(';')[2]
            starActor1 = fullinf.split(';')[3].split(',')[0]
            starActor2 = fullinf.split(';')[3].split(',')[1]
            chrono = fullinf.split(';')[4]
        except IndexError:
            country = 'Неизвестно'
            genre = 'Неизвестно'
            producer = 'Неизвестен'
            starActor1 = 'Неизвестен'
            starActor2 = 'Неизвестен'
            chrono = 'Неизвестно'   
        
        # Тут рейтинг нахожу записываю
        try:
            rating = float(fullinf.split(';')[5])
        except ValueError:
            rating = 0
        except IndexError:
            print(fullinf)
            
        # Добавляю в словарь
        result_list['country'].append(country)
        result_list['genre'].append(genre)
        result_list['producer'].append(producer)
        result_list['starActor1'].append(starActor1)    
        result_list['starActor2'].append(starActor2)
        result_list['chrono'].append(chrono)
        result_list['rating'].append(rating)
        
    pagenum = pagenum + 1 # перехожу на след страницу)
    


CPU times: total: 0 ns
Wall time: 0 ns
https://www.kinonews.ru/top250imdb_p1/
https://www.kinonews.ru/top250imdb_p2/
https://www.kinonews.ru/top250imdb_p3/
https://www.kinonews.ru/top250imdb_p4/
Error

https://www.kinonews.ru/top250imdb_p5/


In [6]:
# Просто вывод длины для проверки
print(len(result_list['idFilm']))
print(len(result_list['titleRU']))
print(len(result_list['titleEN']))
print(len(result_list['yearPub']))
print(len(result_list['country']))
print(len(result_list['genre']))
print(len(result_list['producer']))
print(len(result_list['starActor1']))
print(len(result_list['starActor2']))
print(len(result_list['rating']))
print(len(result_list['chrono']))
print(len(result_list['description']))

250
250
250
250
250
250
250
250
250
250
250
250


In [7]:
df = pd.DataFrame(data=result_list) # создаю датафрейм
pd.set_option('max_colwidth', 100) # устанавливаю параметрии форматирования
df # вывод

,idFilm,titleRU,titleEN,yearPub,description,chrono,country,genre,producer,starActor1,starActor2,rating
0,1,Побег из Шоушенка,The Shawshank Redemption,1994,"""Побег из Шоушенка"" - фильм, который считается одним из лучших в истории кино. Режиссером выступ...",142 мин.,США,драма,Фрэнк Дарабонт,Тим Роббинс,Морган Фриман,9.44
1,2,Крестный отец,The Godfather,1972,"Фильм, ставший классикой своего жанра, рассказывающий историю влиятельного нью-йоркского мафиозн...",175 мин.,США,"драма, триллер",Френсис Форд Коппола,Аль Пачино,Марлон Брандо,9.11
2,3,Темный рыцарь,The Dark Knight,2008,Бэтман вершит правосудие в Готэме. Его партнерами становятся инспектор полиции Гордон и новый ок...,152 мин.,США,"боевик, драма, кинокомикс, криминал, триллер",Кристофер Нолан,Кристиан Бейл,Хит Леджер,9.12
3,4,Крестный отец 2,The Godfather: Part II,1974,Продолжение эпохальной саги режиссера Френсиса Форда Копполы о развитии мафии в Америке. На этот...,202 мин.,США,"драма, криминал",Френсис Форд Коппола,Аль Пачино,Роберт Дюваль,8.88
4,5,12 разгневанных мужчин,12 Angry Men,1957,"Один из 12 присяжных в деле об убийстве пытается убедить остальных присяжных в том, что в деле,...",96 мин.,США,"детектив, драма",Сидни Люмет,Генри Фонда,Джек Уорден,9.16
...,...,...,...,...,...,...,...,...,...,...,...,...
245,246,Хладнокровный Люк,Cool Hand Luke,1967,Упорный и упрямый мужчина отказывается смиряться с тюремными правилами.,127 мин.,США,"драма, криминал",Стюарт Розенберг,Пол Ньюман,Деннис Хоппер,8.00
246,247,Это случилось однажды ночью,It Happened One Night,1934,"Испорченная богатая наследница убегает из дому и неожиданно находит поддержку в лице мужчины, ко...",105 мин.,США,"комедия, мелодрама",Фрэнк Капра,Кларк Гейбл,Клодетт Колбер,8.60
247,248,"Париж, Техас","Paris, Texas",1984,"История уже не молодого человека по имени Трэвис, потерявшего себя на 4 года в американской глуш...",145 мин.,"Германия (ФРГ), Франция, Великобритания, США",драма,Вим Вендерс,Гарри Дин Стэнтон,Дин Стоквел,7.33
248,249,Аладдин,Aladdin,1992,"В сказочном городе Аграба живет Аладдин - вор с добрым сердцем. Но здесь же, во дворце, живет зл...",90 мин.,США,"комедия, мелодрама, мультфильм, мюзикл, приключения, семейный, фэнтези","Рон Клементс, Джон Маскер",Скотт Венгер,Робин Уильямс,8.66


In [8]:
df.to_csv('top_250_cinemas.csv', index=False) # Сохраняю в csv формат

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   idFilm       250 non-null    int64  
 1   titleRU      250 non-null    object 
 2   titleEN      250 non-null    object 
 3   yearPub      250 non-null    int64  
 4   description  250 non-null    object 
 5   chrono       250 non-null    object 
 6   country      250 non-null    object 
 7   genre        250 non-null    object 
 8   producer     250 non-null    object 
 9   starActor1   250 non-null    object 
 10  starActor2   250 non-null    object 
 11  rating       250 non-null    float64
dtypes: float64(1), int64(2), object(9)
memory usage: 23.6+ KB


In [10]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
idFilm,250.0,125.49600,72.309312,1.0,63.250,125.50,187.75,250.00
yearPub,250.0,1987.29200,25.377447,1921.0,1968.500,1995.00,2007.00,2024.00
rating,250.0,7.01064,3.359945,0.0,8.055,8.52,8.81,9.67


In [11]:
df.describe(include=object).T

,count,unique,top,freq
titleRU,250,250,Побег из Шоушенка,1
titleEN,250,248,-,3
description,250,250,"""Побег из Шоушенка"" - фильм, который считается одним из лучших в истории кино. Режиссером выступ...",1
chrono,250,131,130 мин.,8
country,250,54,США,145
genre,250,156,драма,18
producer,250,162,Кристофер Нолан,8
starActor1,250,187,Роберт Де Ниро,7
starActor2,250,223,Мэтт Дэймон,3


In [12]:
df.sample(10)

,idFilm,titleRU,titleEN,yearPub,description,chrono,country,genre,producer,starActor1,starActor2,rating
232,233,Хатико: Самый верный друг,Hachi: A Dog`s Tale,2009,"Фильм о великой преданности и замечательной дружбе снят по одной истории, основанной на реальных...",89 мин.,США,"драма, семейный",Лассе Халльстрем,Джоан Аллен,Ричард Гир,8.71
82,83,Джокер,Joker,2019,"История о том, как Артур Флек, страдающий психическим расстройством и мечтающий работать стендап...",122 мин.,США,"драма, кинокомикс, криминал, приключения, триллер",Тодд Филлипс,Роберт Де Ниро,Хоакин Феникс,8.38
238,239,Битва за Алжир,The Battle of Algiers,1966,Фильм о самой кровавой революции 19-го века.,7.60,"Италия, Алжир","военный, драма, исторический",Джилло Понтекорво,Брахим Хаджади,Жан Мартен,0.00
84,85,Рай и ад,High and Low,1963,"Как близко иной раз беда проходит мимо нас, но многи ли способны, оценив дар судьбы, помочь спра...",8.20,Япония,"драма, криминал, мистика, триллер",Акира Куросава,Тосиро Мифунэ,Тацуя Накадаи,0.00
2,3,Темный рыцарь,The Dark Knight,2008,Бэтман вершит правосудие в Готэме. Его партнерами становятся инспектор полиции Гордон и новый ок...,152 мин.,США,"боевик, драма, кинокомикс, криминал, триллер",Кристофер Нолан,Кристиан Бейл,Хит Леджер,9.12
239,240,Гроздья гнева,The Grapes of Wrath,1940,Во времена Великой депрессии бедная семья из Оклахомы вынуждена покинуть свою ферму и отправитьс...,129 мин.,США,драма,Джон Форд,Генри Фонда,Джейн Дарвелл,8.20
40,41,Леон,Léon,1994,Леон - профессиональный убийца. Он с легкостью выполняет самые сложные заказы и всегда придержив...,110 мин.,Франция,"боевик, драма, криминал, триллер",Люк Бессон,Жан Рено,Гэри Олдман,8.95
75,76,Олдбой,Oldboy,2003,"Похищенный и заключенный в неволю на 15 лет О Дэ-су освобожден. Освобожден только для того, чтоб...",120 мин.,Корея Южная,"боевик, детектив, драма, кинокомикс, триллер",Пак Чхан-ук,Чхве Мин Сик,Цзи Тэ Ю,8.49
152,153,Телохранитель,Yôjinbô,1961,"Япония, незадолго до реставрации Мейдзи. Дорога приводит одинокого самурая без прошлого в мален...",8.50,Япония,"боевик, приключения",Акира Куросава,Тосиро Мифунэ,Тацуя Накадаи,0.00
12,13,Бойцовский клуб,Fight Club,1999,"Обычный клерк, угнетенный своей обыденной жизнью, и торговец, со своим пониманием морали, сталки...",139 мин.,"Германия, США","драма, криминал, триллер",Дэвид Финчер,Брэд Питт,Хелена Бонем Картер,8.90


In [13]:
df[df['idFilm'] == 191]

,idFilm,titleRU,titleEN,yearPub,description,chrono,country,genre,producer,starActor1,starActor2,rating
190,191,Общество мертвых поэтов,Dead Poets Society,1989,"Тодд Андерсон был крайне застенчивым юношей. Родители отправили его учиться в колледж, где старш...",128 мин.,США,драма,Питер Уир,Итан Хоук,Робин Уильямс,8.74
191,191,-,-,1970,-,Неизвестно,Неизвестно,Неизвестно,Неизвестен,Неизвестен,Неизвестен,8.74
